In [12]:
# buffer loop (1st read and write buffer, plus subsequent read and append buffer)

import os
import time
from datetime import datetime
import pandas as pd
from sodapy import Socrata

apptoken = os.environ.get("wMEkdLbVuXIpLiCFVic1PgiZ3")
domain = "data.melbourne.vic.gov.au"
client = Socrata(domain, apptoken)

interval, hours = 15, 1*24*7*6
runs = int(hours*(60/interval))
ds_id, ds_no = 'vh2v-4nfs', "003"
ds_fname = os.path.join('datasets', ds_no+"_"+ds_id+"__bufferred"+".csv")
uq_fname = os.path.join('datasets', ds_no+"_"+ds_id+"__uniqueBays"+".csv")
ds_col1 = ['st_marker_id', 'status']
ds_col2 = ['bay_id', 'st_marker_id', 'lat', 'lon']
log_fname = os.path.join('datasets', ds_no+"_"+ds_id+"__log"+".txt")
for i in range(runs):
    r1 = client.get_all(ds_id)  # get snapshot of dataset using sodapy api
    df_temp = pd.DataFrame.from_dict(r1)  # read dict to dataframe
    df_temp1 = df_temp[ds_col1]
    df_temp2 = df_temp[ds_col2]
    df_temp1['db_read_time'] = datetime.now()  # add timestampe column
    if i == 0:
        df_temp1.to_csv(ds_fname, mode='w', header=True, index=False)  # first write
        df_temp2.to_csv(uq_fname, mode='w', header=True, index=False)
        o = open(log_fname,'w')
        print(f"{datetime.now()}, dataset retrieved and written to csv", file=o)
        print(f"{datetime.now()}, unique parking bays csv written", file=o)
        o.close()
    else:
        df_temp1.to_csv(ds_fname, mode='a', header=False, index=False)  # subsequent append to csv
        df_temp2prev = pd.read_csv(uq_fname)
        df_temp2new = pd.concat([df_temp2prev, df_temp2], axis='index').drop_duplicates(subset=['st_marker_id'])
        df_temp2new.to_csv(uq_fname, mode='w', header=True, index=False)
        o = open(log_fname,'a')
        print(f"{datetime.now()}, dataset retrieved and appended to csv", file=o)
        print(f"{datetime.now()}, unique parking bays csv updated", file=o)
        o.close()
    time.sleep(60*interval)  # sleep 15min before next read and append

<ipython-input-12-be1d06d613cf>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp1['db_read_time'] = datetime.now()  # add timestampe column


In [ ]:
# only read and write 1st buffer

import os
import time
from datetime import datetime
import pandas as pd
from sodapy import Socrata

apptoken = os.environ.get("wMEkdLbVuXIpLiCFVic1PgiZ3")
domain = "data.melbourne.vic.gov.au"
client = Socrata(domain, apptoken)

ds_id, ds_no = 'vh2v-4nfs', "003"
ds_fname = os.path.join('datasets', ds_no+"_"+ds_id+"__bufferred"+".csv")
uq_fname = os.path.join('datasets', ds_no+"_"+ds_id+"__uniqueBays"+".csv")
ds_col1 = ['st_marker_id', 'status']
ds_col2 = ['bay_id', 'st_marker_id', 'lat', 'lon']
log_fname = os.path.join('datasets', ds_no+"_"+ds_id+"__log"+".txt")

r1 = client.get_all(ds_id)  # get snapshot of dataset using sodapy api
df_temp = pd.DataFrame.from_dict(r1)  # read dict to dataframe
df_temp1 = df_temp[ds_col1]
df_temp2 = df_temp[ds_col2]
df_temp1['db_read_time'] = datetime.now()  # add timestampe column

df_temp1.to_csv(ds_fname, mode='w', header=True, index=False)  # first write
df_temp2.to_csv(uq_fname, mode='w', header=True, index=False)
o = open(log_fname,'w')
print(f"{datetime.now()}, dataset retrieved and written to csv", file=o)
print(f"{datetime.now()}, unique parking bays csv written", file=o)
o.close()

In [ ]:
# read and write subsequent append buffers (exclude 1st read and write)

import os
import time
from datetime import datetime
import pandas as pd
from sodapy import Socrata

apptoken = os.environ.get("wMEkdLbVuXIpLiCFVic1PgiZ3")
domain = "data.melbourne.vic.gov.au"
client = Socrata(domain, apptoken)

interval, hours = 15, 1*24*7*6
runs = int(hours*(60/interval))
ds_id, ds_no = 'vh2v-4nfs', "003"
ds_fname = os.path.join('datasets', ds_no+"_"+ds_id+"__bufferred"+".csv")
uq_fname = os.path.join('datasets', ds_no+"_"+ds_id+"__uniqueBays"+".csv")
ds_col1 = ['st_marker_id', 'status']
ds_col2 = ['bay_id', 'st_marker_id', 'lat', 'lon']
log_fname = os.path.join('datasets', ds_no+"_"+ds_id+"__log"+".txt")
for i in range(runs):
    r1 = client.get_all(ds_id)  # get snapshot of dataset using sodapy api
    df_temp = pd.DataFrame.from_dict(r1)  # read dict to dataframe
    df_temp1 = df_temp[ds_col1]
    df_temp2 = df_temp[ds_col2]
    df_temp1['db_read_time'] = datetime.now()  # add timestampe column

    df_temp1.to_csv(ds_fname, mode='a', header=False, index=False)  # subsequent append to csv
    df_temp2prev = pd.read_csv(uq_fname)
    df_temp2new = pd.concat([df_temp2prev, df_temp2], axis='index').drop_duplicates(subset=['st_marker_id'])
    df_temp2new.to_csv(uq_fname, mode='w', header=True, index=False)
    o = open(log_fname,'a')
    print(f"{datetime.now()}, dataset retrieved and appended to csv", file=o)
    print(f"{datetime.now()}, unique parking bays csv updated", file=o)
    o.close()
    
    time.sleep(60*interval)  # sleep 15min before next read and append

In [2]:
# read and write subsequent append buffers (exclude 1st read and write)

import os
import time
from datetime import datetime
import pandas as pd
from sodapy import Socrata
import pyarrow.parquet as pq
import pyarrow as pa

apptoken = "wMEkdLbVuXIpLiCFVic1PgiZ3"
domain = "data.melbourne.vic.gov.au"
client = Socrata(domain, apptoken)

In [ ]:
ds_col1 = ['bay_id', 'st_marker_id', 'status', 'lat', 'lon']
ds_col2 = ['bay_id', 'st_marker_id', 'lat', 'lon']

r1 = client.get_all(ds_id)  # get snapshot of dataset using sodapy api
df_temp = pd.DataFrame.from_dict(r1)  # read dict to dataframe
df_temp1 = df_temp[ds_col1].copy()
df_temp2 = df_temp[ds_col2].copy()
df_temp1['db_read_time'] = datetime.now()  # add timestampe column


In [5]:
df = pd.read_csv('datasets/003_vh2v-4nfs__bufferred.csv')

In [12]:
df.to_parquet('datasets/003_vh2v-4nfs__bufferred.parquet.gzip', compression='gzip')

In [12]:
dfg = pd.read_parquet('datasets/029_d6mv-s43h__bufferred.parquet')

In [15]:
dfg.tail()

,sensor_id,direction_1,direction_2,date_time
3228,77,2,0,2021-09-30T11:09:00.000
3229,25,4,5,2021-09-30T10:59:00.000
3230,69,0,1,2021-09-30T11:06:00.000
3231,53,2,6,2021-09-30T11:01:00.000
3232,26,3,3,2021-09-30T10:58:00.000


In [7]:
dfp = pd.read_parquet('datasets/003_vh2v-4nfs__bufferred.parquet')

In [10]:
dfp.tail()

,bay_id,st_marker_id,status,lat,lon,db_read_time
1344,3896,C5704,Unoccupied,-37.806092441005,144.95511145791926,2021-09-30 11:46:08.302565
1345,2247,3936N,Present,-37.81224916081595,144.95443640358283,2021-09-30 11:46:08.302565
1346,4248,C6828,Present,-37.80938773145017,144.95563994568715,2021-09-30 11:46:08.302565
1347,2691,10200E,Unoccupied,-37.803528781966094,144.94963406770725,2021-09-30 11:46:08.302565
1348,3811,C5458,Unoccupied,-37.807448011270026,144.95443689661118,2021-09-30 11:46:08.302565


In [11]:
dfb.to_parquet('datasets/003_vh2v-4nfs__bufferred.parquet.brotli', compression='brotli')

In [3]:
dfcl = pd.read_csv('datasets/031_h57g-5234__list.csv')

In [7]:
dfcl.loc[~dfcl.note.isnull()]

,sensor_id,sensor_description,sensor_name,installation_date,status,direction_1,direction_2,latitude,longitude,location,note
9,25,Melbourne Convention Exhibition Centre,MCEC_T,2019-10-02T00:00:00.000,A,East,West,-37.824018,144.956044,"{'latitude': '-37.82401776', 'longitude': '144...",Sensor relocated from 14 to 25 on 2/10/2019
10,60,Flinders La - Swanston St (West) Temporary,Swa31T_T,2019-03-08T00:00:00.000,R,North,South,-37.816682,144.966895,"{'latitude': '-37.81668188', 'longitude': '144...",Temporary for the duration of the metro tunnel...
12,39,Alfred Place,AlfPl_T,2019-12-04T00:00:00.000,A,South,North,-37.813797,144.969957,"{'latitude': '-37.81379749', 'longitude': '144...",In 4/12/2019 sensor upgraded
13,17,Collins Place (South),Col15_T,2009-03-30T00:00:00.000,A,East,West,-37.813625,144.973236,"{'latitude': '-37.81362542', 'longitude': '144...",Device is upgraded in 26/02/2020
22,29,St Kilda Rd-Alexandra Gardens,AG_T,2013-10-11T00:00:00.000,A,North,South,-37.819658,144.968635,"{'latitude': '-37.81965808', 'longitude': '144...",sensor upgraded from laser to 3D on 19/12/2019
31,38,Flinders St-Swanston St (West),Swa11_T,2014-12-05T00:00:00.000,R,NaN,NaN,-37.817234,144.967150,"{'latitude': '-37.81723437', 'longitude': '144...",Device has been removed (17/02/2017)
32,32,City Square,CSq_T,2013-12-20T00:00:00.000,R,NaN,NaN,-37.815737,144.966857,"{'latitude': '-37.81573742', 'longitude': '144...",Device has been removed (24/01/2017)
57,16,Australia on Collins,Col270_T,2009-03-30T00:00:00.000,R,NaN,NaN,-37.815734,144.965210,"{'latitude': '-37.81573423', 'longitude': '144...",Device moved to location ID 53 (22/09/2015)
75,35,Southbank,SouthB_T,2021-04-15T00:00:00.000,A,East,West,-37.820178,144.965089,"{'latitude': '-37.82017828', 'longitude': '144...",River side
76,6,Flinders Street Station Underpass,FliS_T,2009-03-25T00:00:00.000,A,North,South,-37.819117,144.965583,"{'latitude': '-37.81911704', 'longitude': '144...",Upgraded on 8/09/21
